<a href="https://colab.research.google.com/github/razanemeguellati/Tp_Cuda_HPC/blob/main/Tp_Cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP2 HPC : Programmation des GPUs NVIDIA-CUDA
Il est demandé de réaliser le produit de deux matrices de taille 256*256 sur le GPU fournit
par l&#39;environnement Google Colab. A cet effet, il est nécessaire de travailler avec une seule
grille de taille 16*16 (256 blocs organisés en 2 dimensions) et des blocs à 2 dimensions de
taille 4*4.

1. Proposer ci-dessous le code CUDA C permettant de réaliser le produit de deux matrices
(C = A x B) en mobilisant la grille et en faisant travailler un thread par bloc. Ainsi, chaque
thread devra calculer une ligne de C.
2. Proposer ci-dessous un autre code CUDA C permettant de réaliser le produit de deux
matrices (C = A x B) en mobilisant la grille et en faisant travailler tous les threads par bloc.
Ainsi, chaque bloc de threads devra générer une ligne de C.

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-glx7mk09
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-glx7mk09
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#define N 256
#define GRID_SIZE 16
#define BLOCK_SIZE 4

__global__ void produit_matrice_matrice(int *a , int *b , int *c)
{
  int i = blockIdx.y*GRID_SIZE + blockIdx.x;

   int somme = 0 ;

       for (int k = 0; k < N; k ++)
        {
          for(int j = 0 ; j< N; j ++ )
          {
              somme += a[i * N + j] * b[ j * N + k ];
          }
          c[i * N + k] = somme ;
          somme=0 ;
        }
}


int main()
{
    int *a , *b , *c ;
    int *gpu_a, *gpu_b, *gpu_c;
    size_t size = N * N * sizeof(int);

    cudaMalloc((void **)&gpu_a, size);
    cudaMalloc((void **)&gpu_b, size);
    cudaMalloc((void **)&gpu_c, size);

    a = (int *) malloc (size);
    b = (int *) malloc (size);
    c = (int *) malloc (size);

   for (int i = 0; i < N*N; i++) {
      a[i] = 1 ;
      b[i] = 1 ;
   }

    cudaMemcpy(gpu_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_b, b, size, cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    float elapsedTime;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start,0);


    dim3 grid(GRID_SIZE, GRID_SIZE);
    // Invocation du kernel avec nb blocs = nb blocs dans la grille , de 1 thread(car on mobilise q'un seul thread)
    produit_matrice_matrice<<<grid,1>>>(gpu_a, gpu_b, gpu_c);



  cudaEventRecord(stop,0);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&elapsedTime, start,stop);
  printf("Elapsed time : %f ms\n" ,elapsedTime);


    cudaMemcpy(c, gpu_c, size, cudaMemcpyDeviceToHost);


   printf("Result Matrix:\n");

    for (int i = 0; i < N; ++i) {
    for (int j = 0; j < N; ++j) {
        printf("%15d ", c[i * N + j]);
    }
    printf("\n");
    }


    cudaFree(gpu_a);
    cudaFree(gpu_b);
    cudaFree(gpu_c);

    free(a);
    free(b);
    free(c);

    return 0;
}


Elapsed time : 3.681280 ms
Result Matrix:
            256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             25

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#define N 256
#define GRID_SIZE 16
#define BLOCK_SIZE 4

__global__ void produit_matrice_matrice(int *a , int *b , int *c)
{
        int i = blockIdx.y*GRID_SIZE + blockIdx.x;
        int numThread = threadIdx.y * BLOCK_SIZE + threadIdx.x;

         for (int k = 0; k < 16 ; k ++)
         {
            int somme = 0  ;
            for(int j = 0 ; j < N ; j ++ )
            {
                somme += a [i * N + j ] * b [j * N + k + numThread * BLOCK_SIZE];

            }
           c[i * N + k + numThread * BLOCK_SIZE] = somme;

        }

}


int main()
{
    int *a , *b , *c ;
    int *gpu_a, *gpu_b, *gpu_c;
    size_t size = N * N * sizeof(int);

    cudaMalloc((void **)&gpu_a, size);
    cudaMalloc((void **)&gpu_b, size);
    cudaMalloc((void **)&gpu_c, size);

    a = (int *) malloc (size);
    b = (int *) malloc (size);
    c = (int *) malloc (size);

   for (int i = 0; i < N*N; i++) {
      a[i] = 1 ;
      b[i] = 1 ;
   }

    cudaMemcpy(gpu_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_b, b, size, cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    float elapsedTime;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start,0);


    dim3 grid(GRID_SIZE, GRID_SIZE);
    dim3 block (BLOCK_SIZE, BLOCK_SIZE);
    // Invocation du kernel avec nb blocs = nb blocs dans la grille , de 1 thread(car on mobilise q'un seul thread)

    produit_matrice_matrice<<<grid,block>>>(gpu_a, gpu_b, gpu_c);



  cudaEventRecord(stop,0);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&elapsedTime, start,stop);
  printf("Elapsed time : %f ms\n" ,elapsedTime);


    cudaMemcpy(c, gpu_c, size, cudaMemcpyDeviceToHost);


   printf("Result Matrix:\n");

    for (int i = 0; i < N; ++i) {
    for (int j = 0; j < N; ++j) {
        printf("%15d ", c[i * N + j]);
    }
    printf("\n");
    }

    cudaFree(gpu_a);
    cudaFree(gpu_b);
    cudaFree(gpu_c);

    free(a);
    free(b);
    free(c);

    return 0;
}


Elapsed time : 0.585568 ms
Result Matrix:
            256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             256             25